In [ ]:
import panel
import param
import pandas as pd

panel.extension()

In [ ]:
import sys

sys.path.insert(0, "../dags/")
from jobs.common.bigquery import get_table_as_records
from google.cloud import bigquery
from google.oauth2 import service_account

key_path = "/Users/yco/.dbt/dbt-user-creds.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path  # , scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(
    credentials=credentials,
    project=credentials.project_id,
)
perfs = pd.DataFrame(
    get_table_as_records(
        client,
        "model_perfs",
        "textcat",
    )
)

In [ ]:
perfs

In [ ]:
# def sanitize(d):
#     return {
#         k: str(v)
#         if isinstance(v, dict)
#         else v.strftime("%Y-%m-%d")
#         if k == "date"
#         else v
#         for k, v in d.items()
#     }

# new_data = []
# for _, data in perfs.to_dict(orient="index").items():
#     real_dat = data["valid"] if data["type"] == "valid" else data["train"]
#     data.pop("valid")
#     data.pop("train")
#     if real_dat is None:
#         new_data.append(sanitize(data))
#     else:
#         new_data.append(sanitize({**data, **real_dat}))
# job_config = bigquery.LoadJobConfig()
# job_config.autodetect = True
# job_config.schema_update_options = [
#     "ALLOW_FIELD_ADDITION",
#     "ALLOW_FIELD_RELAXATION",
# ]
# job_config.write_disposition = "WRITE_APPEND"
# table = "model_perfs.textcat"
# client.load_table_from_json(
#     new_data, table, job_config=job_config
# ).result()

In [ ]:
import holoviews as hv

hv.extension("bokeh")


class ModelPerfs(param.Parameterized):
    perf_metric = param.Selector(
        objects=["AUC", "score", "micro_prf", "macro_prf"]
    )  # TODO: PRF per label
    perf_type = param.Selector(objects=["valid", "train"])
    model = param.Selector(objects=perfs.model.unique())

    def view(self):
        if self.perf_metric == "AUC":
            variables = ["cats_macro_auc"]
        elif self.perf_metric == "score":
            variables = ["cats_score"]
        elif self.perf_metric == "micro_prf":
            variables = ["cats_micro_p", "cats_micro_r", "cats_micro_f"]
        elif self.perf_metric == "macro_prf":
            variables = ["cats_macro_p", "cats_macro_r", "cats_macro_f"]
        cols = ["date", "model", *variables]
        df = (
            perfs.loc[perfs.type == self.perf_type, cols]
            .set_index(["date", "model"])
            .stack()
            .sort_index()
        )
        df.name = "performance"
        df.index.names = df.index.names[:2] + ["variables"]
        df.reset_index()
        ds = hv.Dataset(
            df,
            kdims=["date", "model", "variables"],
            vdims=hv.Dimension("performance", range=(0, 1)),
        )
        return (
            ds.to(hv.Curve, "date")
            .overlay("variables")
            .opts(legend_position="right", toolbar="above", width=600)
        )


obj = ModelPerfs()
panel.Row(obj.param, obj.view)